## Setup

In [1]:
import sys
import os
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd()))

from sqlalchemy import create_engine
from utils import get_session
from data.models.eventhub import Ticket
from agentic.agents.classifier import TicketClassifier, classify_ticket
from agentic.agents.prompts.classifier_prompts import TICKET_CATEGORIES, TICKET_URGENCY, TICKET_SENTIMENT

import pandas as pd

# Create database engine
DB_PATH = "data/db/eventhub.db"
engine = create_engine(f"sqlite:///{DB_PATH}")

## View Available Categories

In [2]:
print("Available Ticket Categories:\n")
for category, description in TICKET_CATEGORIES.items():
    print(f"• {category.upper()}: {description}")

print("\n\nUrgency Levels:\n")
for urgency, description in TICKET_URGENCY.items():
    print(f"• {urgency.upper()}: {description}")

print("\n\nSentiment:\n")
for sentiment, description in TICKET_SENTIMENT.items():
    print(f"• {sentiment.upper()}: {description}")

Available Ticket Categories:

• REFUND: Refund requests for tickets or event cancellations
• CANCELLATION: Customer wants to cancel reservations or bookings
• GENERAL: General inquiries about events, accessibility, parking, memberships
• TECHNICAL: Website, app, login, payment processing, or technical platform issues
• COMPLAINT: Customer service issues, venue problems, quality complaints


Urgency Levels:

• LOW: General questions, not time-sensitive
• MEDIUM: Standard issues, can wait a few hours
• HIGH: Important issues affecting ability to attend event
• CRITICAL: Event today, payment failures, account locked


Sentiment:

• POSITIVE: Happy, grateful, satisfied
• NEUTRAL: Just asking questions, no strong emotion
• NEGATIVE: Frustrated, angry, upset, disappointed


## Load Sample Tickets from Database

In [3]:
# Get database session using context manager
with get_session(engine) as session:
    # Load a sample of tickets (first 20)
    tickets_query = session.query(Ticket).limit(20).all()
    
    # Extract data before session closes to avoid DetachedInstanceError
    tickets = []
    for ticket in tickets_query:
        tickets.append({
            'ticket_id': ticket.ticket_id,
            'category': ticket.category,
            'subject': ticket.subject,
            'description': ticket.description,
            'status': ticket.status,
            'user_email': ticket.user_email,
            'priority': ticket.priority
        })
    
    print(f"Loaded {len(tickets)} tickets from database\n")
    
    # Display first 3 tickets with their actual categories
    print("Sample Tickets:")
    for i, ticket in enumerate(tickets[:3], 1):
        print(f"\n--- Ticket {i} (ID: {ticket['ticket_id']}) ---")
        print(f"Actual Category: {ticket['category']}")
        print(f"Subject: {ticket['subject']}")
        print(f"Description: {ticket['description'][:150]}...")
        print(f"Status: {ticket['status']}")

Loaded 20 tickets from database

Sample Tickets:

--- Ticket 1 (ID: t_00001) ---
Actual Category: general
Subject: Accessibility information for Comedy Night
Description: Hi, I'm attending The Comedy Night: The Tech Takeover on the 20th of this month with my friend. We both require wheelchair access. Could you please co...
Status: open

--- Ticket 2 (ID: t_00002) ---
Actual Category: general
Subject: Group booking discount for comedy event
Description: I'd like to book a table for 10 people for The Comedy Night: The Tech Takeover on May 5th. Do you offer group booking discounts and if so, what's the ...
Status: in_progress

--- Ticket 3 (ID: t_00003) ---
Actual Category: general
Subject: Premium membership benefits for comedy show
Description: I recently purchased a premium membership. I'm excited to attend The Comedy Night: The Tech Takeover on April 10th. Could you please remind me of the ...
Status: in_progress


## Test Single Ticket Classification

In [4]:
# Initialize classifier
classifier = TicketClassifier()

# Test with first ticket
test_ticket = tickets[0]
print(f"Testing Ticket ID: {test_ticket['ticket_id']}")
print(f"Actual Category: {test_ticket['category']}")
print(f"Subject: {test_ticket['subject']}")
print(f"Description: {test_ticket['description'][:200]}...\n")

# Classify
result = classifier.classify(
    subject=test_ticket['subject'],
    description=test_ticket['description'],
    ticket_id=test_ticket['ticket_id']
)

# Display results
print("\n=== Classification Result ===")
print(f"Category: {result['category'].upper()} (Actual: {test_ticket['category']})")
print(f"Urgency: {result['urgency'].upper()}")
print(f"Sentiment: {result['sentiment'].upper()}")
print(f"Summary: {result['summary']}")
print(f"\nMatch: {'✅' if result['category'] == test_ticket['category'] else '❌'}")

Testing Ticket ID: t_00001
Actual Category: general
Subject: Accessibility information for Comedy Night
Description: Hi, I'm attending The Comedy Night: The Tech Takeover on the 20th of this month with my friend. We both require wheelchair access. Could you please confirm if The Laughing Avocado is wheelchair access...


=== Classification Result ===
Category: GENERAL (Actual: general)
Urgency: MEDIUM
Sentiment: NEUTRAL
Summary: Customer inquiring about wheelchair accessibility for The Comedy Night event

Match: ✅


## Test Batch Classification

In [5]:
# Prepare batch of tickets
batch_tickets = [
    {
        "ticket_id": ticket['ticket_id'],
        "subject": ticket['subject'],
        "description": ticket['description']
    }
    for ticket in tickets[:10]  # First 10 tickets
]

print(f"Classifying {len(batch_tickets)} tickets...\n")

# Classify batch
results = classifier.classify_batch(batch_tickets)

# Display results in table
results_df = pd.DataFrame([
    {
        "Ticket ID": r["ticket_id"],
        "Actual": tickets[i]['category'],
        "Predicted": r["category"],
        "Urgency": r["urgency"],
        "Sentiment": r["sentiment"],
        "Match": "✅" if r["category"] == tickets[i]['category'] else "❌"
    }
    for i, r in enumerate(results)
])

print(results_df.to_string(index=False))

Classifying 10 tickets...

Ticket ID  Actual Predicted Urgency Sentiment Match
  t_00001 general   general  medium   neutral     ✅
  t_00002 general   general     low   neutral     ✅
  t_00003 general   general     low  positive     ✅
  t_00004 general   general     low   neutral     ✅
  t_00005 general   general     low   neutral     ✅
  t_00006 general   general     low   neutral     ✅
  t_00007 general   general     low   neutral     ✅
  t_00008 general   general  medium   neutral     ✅
  t_00009 general   general  medium   neutral     ✅
  t_00010 general   general     low   neutral     ✅


## Analyze Category Distribution

In [6]:
# Get category distribution
distribution = classifier.get_category_distribution(results)

print("\n=== Category Distribution ===")
for category, count in sorted(distribution.items(), key=lambda x: x[1], reverse=True):
    if count > 0:
        print(f"{category.upper()}: {count} tickets")

# Calculate statistics
total_tickets = len(results)
matches = sum(1 for i, r in enumerate(results) if r["category"] == tickets[i]['category'])
accuracy = matches / total_tickets

# Urgency distribution
urgency_dist = {}
for r in results:
    urgency = r["urgency"]
    urgency_dist[urgency] = urgency_dist.get(urgency, 0) + 1

# Sentiment distribution  
sentiment_dist = {}
for r in results:
    sentiment = r["sentiment"]
    sentiment_dist[sentiment] = sentiment_dist.get(sentiment, 0) + 1

print(f"\n=== Statistics ===")
print(f"Total Tickets: {total_tickets}")
print(f"Accuracy: {matches}/{total_tickets} ({accuracy:.1%})")

print(f"\n=== Urgency Distribution ===")
for urgency, count in sorted(urgency_dist.items(), key=lambda x: x[1], reverse=True):
    print(f"{urgency.upper()}: {count} tickets")

print(f"\n=== Sentiment Distribution ===")
for sentiment, count in sorted(sentiment_dist.items(), key=lambda x: x[1], reverse=True):
    print(f"{sentiment.upper()}: {count} tickets")


=== Category Distribution ===
GENERAL: 10 tickets

=== Statistics ===
Total Tickets: 10
Accuracy: 10/10 (100.0%)

=== Urgency Distribution ===
LOW: 7 tickets
MEDIUM: 3 tickets

=== Sentiment Distribution ===
NEUTRAL: 9 tickets
POSITIVE: 1 tickets


## Test Custom Ticket Examples

In [7]:
# Test with custom examples
test_cases = [
    {
        "subject": "Need refund for cancelled event",
        "description": "The Taylor Swift concert was cancelled and I want my money back. I paid $250 for two tickets."
    },
    {
        "subject": "Can't access my account",
        "description": "I forgot my password and the reset link isn't working. I've tried multiple times."
    },
    {
        "subject": "Double charged for tickets",
        "description": "I was charged twice for the same order. Order #12345 shows two charges of $120 on my credit card."
    },
    {
        "subject": "Question about upcoming concert",
        "description": "Is the Coldplay concert on March 15th still happening? I haven't received any updates."
    },
    {
        "subject": "Terrible customer service experience",
        "description": "I've been waiting on hold for 45 minutes and no one is helping me. This is ridiculous!"
    }
]

print("=== Testing Custom Examples ===\n")

for i, test_case in enumerate(test_cases, 1):
    result = classifier.classify(
        subject=test_case["subject"],
        description=test_case["description"]
    )
    
    print(f"\n--- Test Case {i} ---")
    print(f"Subject: {test_case['subject']}")
    print(f"→ Category: {result['category'].upper()}")
    print(f"→ Urgency: {result['urgency'].upper()}")
    print(f"→ Sentiment: {result['sentiment'].upper()}")
    print(f"→ Summary: {result['summary']}")

=== Testing Custom Examples ===


--- Test Case 1 ---
Subject: Need refund for cancelled event
→ Category: REFUND
→ Urgency: MEDIUM
→ Sentiment: NEGATIVE
→ Summary: Customer requesting refund for cancelled Taylor Swift concert

--- Test Case 2 ---
Subject: Can't access my account
→ Category: TECHNICAL
→ Urgency: MEDIUM
→ Sentiment: NEGATIVE
→ Summary: Customer unable to access account due to password reset issue

--- Test Case 3 ---
Subject: Double charged for tickets
→ Category: REFUND
→ Urgency: MEDIUM
→ Sentiment: NEGATIVE
→ Summary: Customer was double charged for the same ticket order

--- Test Case 4 ---
Subject: Question about upcoming concert
→ Category: GENERAL
→ Urgency: MEDIUM
→ Sentiment: NEUTRAL
→ Summary: Customer inquiring about the status of the Coldplay concert on March 15th

--- Test Case 5 ---
Subject: Terrible customer service experience
→ Category: COMPLAINT
→ Urgency: HIGH
→ Sentiment: NEGATIVE
→ Summary: Customer had a poor experience with long hold times and no 

## Test Convenience Function

In [8]:
# Test quick classify_ticket function
quick_result = classify_ticket(
    subject="Can I cancel my reservation?",
    description="I need to cancel my booking for the Jazz Festival on Saturday."
)

print("Quick Classification Test:")
print(f"Category: {quick_result['category']}")
print(f"Urgency: {quick_result['urgency']}")
print(f"Sentiment: {quick_result['sentiment']}")
print(f"Summary: {quick_result['summary']}")

Quick Classification Test:
Category: cancellation
Urgency: medium
Sentiment: neutral
Summary: Customer requests to cancel their Jazz Festival booking


## Summary

The classifier successfully:
- ✅ Classifies tickets into 5 categories (refund, cancellation, general, technical, complaint)
- ✅ Assigns urgency levels (low, medium, high, critical)
- ✅ Detects sentiment (positive, neutral, negative)
- ✅ Generates summary of each ticket
- ✅ Handles batch classification
- ✅ Uses simple, concise prompts (following solution pattern)
- ✅ Validates against actual ticket categories in database

**Next Steps:**
1. Implement Resolver Agent (uses RAG + DB tools)
2. Implement Escalation Agent
3. Build Supervisor/Router
4. Create LangGraph workflow